In [11]:
from rapidfuzz import process
import pandas as pd

# Load the Excel file
file_path = "StateData.xlsx"
sheet_name = "StateData"
df = pd.read_excel(file_path, sheet_name=sheet_name)

# Define a list of standard state names
standard_names = ['Pennsylvania', 'California', 'New York', 'Texas', 'Florida']

# Function to clean state names using fuzzy matching
def clean_name(name):
    result = process.extractOne(name, standard_names, score_cutoff=80)  # Adjust threshold as needed
    if result:  # Check if a match was found
        match, score, _ = result
        return match
    return name

# Clean the State Name column
df['Cleaned State Name'] = df['State Name'].apply(clean_name)

# Save the cleaned data to a new Excel file
output_path = "Cleaned_StateData.xlsx"
df.to_excel(output_path, index=False)

print(f"Cleaned data saved to {output_path}")


Cleaned data saved to Cleaned_StateData.xlsx


It is cleaning most of the names but missing some states with Abbreviation.

# Option 1 Correct but not too optimised

In [8]:
from rapidfuzz import process
import pandas as pd

# Load the Excel file
file_path = "StateData.xlsx"
sheet_name = "StateData"
df = pd.read_excel(file_path, sheet_name=sheet_name)

# Dictionary for abbreviations and common variations
state_dict = {
    'Pennsylvania': ['Pennsylvania', 'PA', 'Penns', 'Pennyslylvania'],
    'California': ['California', 'CA', 'Cal'],
    'New York': ['New York', 'NY', 'N York'],
    'Texas': ['Texas', 'TX', 'Tex'],
    'Florida': ['Florida', 'FL', 'Fla'],
    'Massachusetts' : ['Massachusetts', 'MA', 'Mass']
}

# Flatten the dictionary to create a lookup table
lookup = {alias: state for state, aliases in state_dict.items() for alias in aliases}

# Function to clean state names using dictionary lookup and fuzzy matching
def clean_name(name):
    # Check for exact match using the dictionary
    if name in lookup:
        return lookup[name]
    
    # Fuzzy matching as a fallback
    result = process.extractOne(name, list(state_dict.keys()), score_cutoff=80)
    if result:
        match, score, _ = result
        return match
    return name

# Clean the State Name column
df['Cleaned State Name'] = df['State Name'].apply(clean_name)

# Save the cleaned data to a new Excel file
output_path = "Cleaned_StateData1.xlsx"
df.to_excel(output_path, index=False)

print(f"Cleaned data saved to {output_path}")


Cleaned data saved to Cleaned_StateData1.xlsx


# Option2 : Considering Special characters

In [27]:
from rapidfuzz import process, fuzz
import pandas as pd
import re

# Load the Excel file
file_path = "StateData.xlsx"
sheet_name = "StateData"
df = pd.read_excel(file_path, sheet_name=sheet_name)

# Dictionary for abbreviations and common variations
state_dict = {
    'Pennsylvania': ['Pennsylvania', 'PA', 'Penns', 'Pennyslylvania'],
    'California': ['California', 'CA', 'Cal'],
    'New York': ['New York', 'NY', 'N York'],
    'Texas': ['Texas', 'TX', 'Tex'],
    'Florida': ['Florida', 'FL', 'Fla'],
    'Massachusetts': ['Massachusetts', 'MA', 'Mass'],
    'Illinois': ['Illinois','IL']
}

# Flatten the dictionary to create a lookup table
lookup = {alias: state for state, aliases in state_dict.items() for alias in aliases}

# Function to clean state names using dictionary lookup and fuzzy matching
def preprocess_name(name):
    # Remove special characters, convert to lowercase, and strip spaces
    name = re.sub(r"[^a-zA-Z0-9]", "", name)
    return name

# Prefix mapping to ensure more accurate matches
prefix_dict = {
    'pen': 'Pennsylvania',
    'mas': 'Massachusetts',
    'cal': 'California',
    'tex': 'Texas',
    'flo': 'Florida'
}

def clean_name(name):
    # Preprocess the name (remove special characters and convert to lowercase)
    cleaned_name = preprocess_name(name)
    
    # Check for exact match using the dictionary
    if cleaned_name in lookup:
        return lookup[cleaned_name]
    
    # Prefix matching as an additional condition
    for prefix, state in prefix_dict.items():
        if cleaned_name.lower().startswith(prefix):
            return state
    
    # Fuzzy matching as a fallback
    result = process.extractOne(
        cleaned_name, 
        list(state_dict.keys()), 
        scorer=fuzz.token_sort_ratio, 
        score_cutoff=40
    )
    
    if result:
        match, score, _ = result
        # Ensure the first character matches
        if match[0].lower() == cleaned_name[0].lower():
            # If the matched state is in the dictionary keys, return it
            if match[0] in state_dict:
                return match[0]
    
    # If no match is found or the match isn't a valid state, return blank
    return ""

# Clean the State Name column
df['Cleaned State Name'] = df['State Name'].apply(clean_name)

# Save the cleaned data to a new Excel file
output_path = "Cleaned_StateData10.xlsx"
df.to_excel(output_path, index=False)

print(f"Cleaned data saved to {output_path}")


Cleaned data saved to Cleaned_StateData10.xlsx


# Option3: Best accuracy

In [28]:
from rapidfuzz import process, fuzz
import pandas as pd
import re

# Load the Excel file
file_path = "StateData.xlsx"
sheet_name = "StateData"
df = pd.read_excel(file_path, sheet_name=sheet_name)

# Dictionary for abbreviations and common variations
state_dict = {
    'Pennsylvania': ['Pennsylvania', 'PA', 'Penns', 'Pennyslylvania'],
    'California': ['California', 'CA', 'Cal'],
    'New York': ['New York', 'NY', 'N York'],
    'Texas': ['Texas', 'TX', 'Tex'],
    'Florida': ['Florida', 'FL', 'Fla'],
    'Massachusetts': ['Massachusetts', 'MA', 'Mass'],
    'Illinois': ['Illinois','IL']
}

# Flatten the dictionary to create a lookup table
lookup = {alias: state for state, aliases in state_dict.items() for alias in aliases}

# Function to clean state names using dictionary lookup and fuzzy matching
def preprocess_name(name):
    # Remove special characters, convert to lowercase, and strip spaces
    name = re.sub(r"[^a-zA-Z0-9]", "", name)
    return name

# Prefix mapping to ensure more accurate matches
prefix_dict = {
    'pen': 'Pennsylvania',
    'mas': 'Massachusetts',
    'cal': 'California',
    'tex': 'Texas',
    'flo': 'Florida'
}

def clean_name(name):
    # Preprocess the name (remove special characters and convert to lowercase)
    cleaned_name = preprocess_name(name)
    
    # Check for exact match using the dictionary
    if cleaned_name in lookup:
        return lookup[cleaned_name]
    
    # Prefix matching as an additional condition
    for prefix, state in prefix_dict.items():
        if cleaned_name.lower().startswith(prefix):
            return state
    
    # Fuzzy matching as a fallback
    result = process.extractOne(
        cleaned_name, 
        list(state_dict.keys()), 
        scorer=fuzz.token_sort_ratio, 
        score_cutoff=40
    )
    
    if result:
        match, score, _ = result
        # Ensure the first character matches
        if match[0].lower() == cleaned_name[0].lower():
            return match
    return name

# Clean the State Name column
df['Cleaned State Name'] = df['State Name'].apply(clean_name)

# Additional logic to check if Cleaned State Name is in state_dict keys
df['Cleaned State Name'] = df['Cleaned State Name'].apply(
    lambda x: x if x in state_dict else ""
)


# Save the cleaned data to a new Excel file
output_path = "Cleaned_StateData10.xlsx"
df.to_excel(output_path, index=False)

print(f"Cleaned data saved to {output_path}")


Cleaned data saved to Cleaned_StateData10.xlsx


# Lets do the same for Australia states

In [35]:
from rapidfuzz import process, fuzz
import pandas as pd
import re

# Load the Excel file
file_path = "StateData_Australia.xlsx"
sheet_name = "StateData"
df = pd.read_excel(file_path, sheet_name=sheet_name)

# Dictionary for abbreviations and common variations
state_dict = {
    'New South Wales': ['NSW', 'New South Wales', 'south wales'],
    'Queensland': ['QLD', 'Queensland'],
    'South Australia': ['SA', 'South Australia', 'Sou Aus'],
    'Tasmania': ['TAS', 'Tasmania'],
    'Victoria': ['VIC', 'Victoria'],
    'Western Australia': ['WA', 'Western Australia', 'Westn Aus'],
    'Northern Territory': ['NT','Northern Territory'],
    'Australian Capital Territory': ['ACT', 'Australian Capital Territory', 'Aus Cap']
}

# Prefix mapping to ensure more accurate matches
prefix_dict = {
    'New': 'New South Wales',
    'Qu': 'Queensland',
    'So': 'South Australia',
    'Tas': 'Tasmania',
    'Vic': 'Victoria',
    'Wes': 'Western Australia',
    'Nor': 'Northern Territory',
    'Aus': 'Australian Capital Territory'
}


# Flatten the dictionary to create a lookup table
lookup = {alias: state for state, aliases in state_dict.items() for alias in aliases}

# Function to clean state names using dictionary lookup and fuzzy matching
def preprocess_name(name):
    # Remove special characters, convert to lowercase, and strip spaces
    name = re.sub(r"[^a-zA-Z0-9]", "", name)
    return name


def clean_name(name):
    # Preprocess the name (remove special characters and convert to lowercase)
    cleaned_name = preprocess_name(name)
    
    # Check for exact match using the dictionary
    if cleaned_name in lookup:
        return lookup[cleaned_name]
    
    # Prefix matching as an additional condition
    for prefix, state in prefix_dict.items():
        if cleaned_name.lower().startswith(prefix):
            return state
    
    # Fuzzy matching as a fallback
    result = process.extractOne(
        cleaned_name, 
        list(state_dict.keys()), 
        scorer=fuzz.token_sort_ratio, 
        score_cutoff=40
    )
    
    if result:
        match, score, _ = result
        # Ensure the first character matches
        if match[0].lower() == cleaned_name[0].lower():
            return match
    return name

# Clean the State Name column
df['Cleaned State Name'] = df['State Name'].apply(clean_name)

# Additional logic to check if Cleaned State Name is in state_dict keys
df['Cleaned State Name'] = df['Cleaned State Name'].apply(
    lambda x: x if x in state_dict else ""
)


# Save the cleaned data to a new Excel file
output_path = "Cleaned_StateDataAus2.xlsx"
df.to_excel(output_path, index=False)

print(f"Cleaned data saved to {output_path}")

Cleaned data saved to Cleaned_StateDataAus2.xlsx


In [ ]:
Theory:
Fuzzy Logic
Cosine Similarity